# Leonardo F. de Souza

## Analysis of Fit, Portfolio Optimization, Fama-French Factors, and Portfolio Beta Adjustment

#### October, 2022

###### Leonardo.deSouza@coyotes.usd.edu
###### (605)661-3867
-------------------------------------------------------------------------------------------------------------------------------

#### Modules

In [36]:
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
import yfinance as yf
import numpy as np
import plotly.express as px
from plotly.express import *
import plotly.graph_objects as go
import plotly.figure_factory as ff
from dash_bootstrap_templates import load_figure_template
from plotly.subplots import make_subplots
from scipy.optimize import minimize
import getFamaFrenchFactors as gff
from scipy.stats import linregress
import statsmodels.api as sm
import statsmodels.formula.api as smf
from math import e
from scipy import stats
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')
#--------------------------------------
layout = load_figure_template('DARKLY')

#### Data
##### Let's say we have "n" managers in the portfolio, each assigned a weight "w". Let's also assume our benchmark to be the Russell 3000 (using ETF). For a simpler approach, let the following stocks be a represenation of a "manager". Thus the portfolio returns are:

In [37]:
interval = '1mo'
end = dt.datetime.today()
start = end - relativedelta(years=5)

# Let n be 4, and the "managers" equal the following Stocks:
# 'AAPL, META, JNJ, BRK-B'
# The benchmark be: 'IWV' (RUSSELL 3000 ETF (iShares))
# And the "new manager" be: 'AMZN'
Managers = ['AAPL', 'META', 'JNJ', 'BRK-B', 'IWV', 'AMZN']

# Let our Portfolio Value be $100,000,000:
P = 100000000

# Let w be:
w = [20000000, 15000000, 50000000, 15000000] # funding of each manager
w = [weight/P for weight in w] # weights
print(w)
print(sum(w)) # must be 1

Managers = yf.download(Managers, interval=interval, start=start)['Adj Close'] # if available in Excel, use pd.read_excel

df = pd.DataFrame(Managers)
df.dropna(how='all',axis=0, inplace=True)
df.dropna(how='all',axis=1, inplace=True)

#Returns
df = np.log(df) - np.log(df.shift(1))

[0.2, 0.15, 0.5, 0.15]
1.0
[*********************100%***********************]  6 of 6 completed


#### Our portfolio return can be calculated as:

In [38]:
Man = df[['AAPL', 'META', 'JNJ', 'BRK-B']]
Port_Ret = Man*w
Port_Ret = Port_Ret.sum(axis=1)
Port_Ret = pd.DataFrame(Port_Ret)
Port_Ret.columns = ['Portfolio Return']
Port_Ret = Port_Ret[1::]

#### We could then calculate our excess returns relative to the Benchmark:

<div>
<img src='https://cfastudyguide.com/wp-content/uploads/2020/07/image-38.png' style="width:30%; padding-right: 20px; padding-left: 20px; text-align:center"/>
    </div>
<div>
<img src='https://i0.wp.com/www.modviz.com/wp-content/uploads/2020/11/image-3.png?resize=516%2C105&ssl=1' style="width:24%; padding-right: 20px; padding-left: 20px; text-align:center"/>
    </div>
<div>
<img src='https://marketxls.com/wp-content/uploads/2017/06/Sharpe-Ratio-.png' style="width:20%; padding-right: 20px; padding-left: 20px; text-align:center""/>
    </div>

In [41]:
Excess = ((1+Port_Ret['Portfolio Return'])/(1+df['IWV']))-1
Tracking_Error_1 = Excess.std()*np.sqrt(12)
print("Tracking Error:",f"{round(Tracking_Error_1/.0001,2)} bps")

Annualized_Return_1 = ((np.prod(1+Port_Ret['Portfolio Return']))**(12/((Port_Ret['Portfolio Return'].count()))))-1
print("Average Return:",f"{round(Annualized_Return_1*100,2)}%")

#The Std Dev is:
Stdev_1 = (Port_Ret['Portfolio Return'].std()*np.sqrt(12))
print("Stdev:",f"{round(Stdev_1*100,2)}%")

# Benchmark's Annualized Returns:
BMR = ((np.prod(1+df['IWV']))**(12/((df['IWV'].count()))))-1
#Thus:
IR = (Annualized_Return_1-BMR)/(Tracking_Error_1)
print("Information Ratio:",f"{round(IR,2)}")

SP = (Annualized_Return_1-BMR)/(Stdev_1)
print("Sharpe Ratio:",f"{round(SP,2)}")

Risk_Reward = Annualized_Return_1/Stdev_1
print("Reward/Risk:",f"{round(Risk_Reward,2)}")

Tracking Error: 1132.04 bps
Average Return: 7.26%
Stdev: 18.16%
Information Ratio: 0.05
Sharpe Ratio: 0.03
Reward/Risk: 0.4


### Let's take a look at the manager correlation

In [69]:
# Fist let's see how correlated these "managers" are:
test1 = df[['AAPL', 'META', 'JNJ', 'BRK-B', 'AMZN']]
test1 = test1[1::]
t_cor = round(test1.corr(),3)
cor_fig = px.imshow(t_cor, text_auto=True, aspect="auto",
                    labels=dict(color="Correlation"))
cor_fig.update_layout(
    title_text ='Correlation Matrix',
    yaxis_title ='Managers')
cor_fig.update_xaxes(side="top")
cor_fig

### We may also want to compare single manager returns vs. Portfolio:

In [6]:
# Now, let's find the "Beta" adjusted returns (Portfolio Returns vs. New Manager):
#Thus, let "y" and "x" be:
y = Port_Ret['Portfolio Return']
x = df['AMZN']
x = x[1::]
slope, intercept, r_value, p_value, std_err = linregress(x, y)

# Thus our Beta adjusted returns are:
Beta_Adj = slope*x

<div>
    <h6> Appraisal Ratio:</h6>
</div>
<div>
<img src="https://cfastudyguide.com/wp-content/uploads/2020/08/image-100.png" style="width:17%; padding-right: 20px; padding-left: 20px; text-align:center"/>
    </div>

In [67]:
# Comparing the portfolio and the new manager, we have:

#1. Beta Adjusted Manager:
ERB = ((1+Beta_Adj)/(1+df['IWV'][1::]))-1
TEB = ERB.std()*np.sqrt(12)
ARB = ((np.prod(1+Beta_Adj))**(12/((Beta_Adj.count()))))-1
SDB = (Beta_Adj.std()*np.sqrt(12))
IRB = (ARB-BMR)/(TEB)
RRB = ARB/SDB
y1 = Port_Ret['Portfolio Return']
x1 = Beta_Adj
slope1, intercept1, r_value1, p_value1, std_err1 = linregress(x1, y1)
APP1 = (intercept1/std_err1)

#2. Unadjusted:
ERB2 = ((1+x)/(1+df['IWV'][1::]))-1
TEB2 = ERB2.std()*np.sqrt(12)
ARB2 = ((np.prod(1+x))**(12/((x.count()))))-1
SDB2 = (x.std()*np.sqrt(12))
IRB2 = (ARB2-BMR)/(TEB2)
RRB2 = ARB2/SDB2
APP2 = (intercept/std_err)

Comp1 = ["Annualized Returns", "Standard Deviation", "Information Ratio", "Reward/Risk"]
Comp1 = pd.DataFrame(Comp1)
Comp1['Original Portfolio'] = [Annualized_Return_1, Stdev_1, IR, Risk_Reward]
Comp1['Beta Adj. Manager'] = [ARB, SDB, IRB, RRB]
Comp1['Beta Unadj. Manager'] = [ARB2, SDB2, IRB2, RRB2]
Comp1 = Comp1.rename(columns={Comp1.columns[0]: "Metrics"})
Comp1.loc[len(Comp1.index)] = ['Appraisal Ratio', np.nan, APP1, APP2] 
Comp1

,Metrics,Original Portfolio,Beta Adj. Manager,Beta Unadj. Manager
0,Annualized Returns,0.072636,0.042139,0.096072
1,Standard Deviation,0.181557,0.105609,0.330495
2,Information Ratio,0.051204,-0.169728,0.121565
3,Reward/Risk,0.400072,0.399006,0.290690
4,Appraisal Ratio,NaN,0.017867,0.055915


##### As a visual:

In [8]:
figure = go.Figure()
figure.add_trace(go.Scatter(x=Port_Ret.index, y=Port_Ret['Portfolio Return'], name='Original Portfolio'))
figure.add_trace(go.Scatter(x=x.index, y=x, name='New Manager Returns'))
figure.add_trace(go.Scatter(x=Beta_Adj.index, y=Beta_Adj, name='Beta Adj. Manager Returns'))
figure.update_layout(
    title_text ='MTM Returns',
    yaxis_title ='Returns',
    xaxis_title = 'Date')
figure.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=5,
                            label="5Y",
                            step="year",
                            stepmode="backward"),
                        dict(count=3,
                            label="3Y",
                            step="year",
                            stepmode="backward"),
                        dict(count=1,
                            label="YTD",
                            step="year",
                            stepmode="todate"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True,
                ),
                type="date"
            )
        )
figure.update_layout(yaxis_tickformat='.2%')
figure.update_layout(template='plotly_dark',
                  xaxis_rangeselector_font_color='black',
                  xaxis_rangeselector_activecolor='red',
                  xaxis_rangeselector_bgcolor='green',)
figure.update_layout(xaxis=dict(showgrid=False),
                     yaxis=dict(showgrid=False))
figure.show()

In [9]:
figure = go.Figure()
figure.add_trace(go.Scatter(x=Comp1.loc[1, ['Original Portfolio']], 
                            y=Comp1.loc[0, ['Original Portfolio']],
                            name='Portfolio'))
figure.add_trace(go.Scatter(x=Comp1.loc[1, ['Beta Adj. Manager']], 
                            y=Comp1.loc[0, ['Beta Adj. Manager']],
                            name='Beta Ajusted Manager'))
figure.add_trace(go.Scatter(x=Comp1.loc[1, ['Beta Unadj. Manager']], 
                            y=Comp1.loc[0, ['Beta Unadj. Manager']],
                            name='Unadjusted Manager'))
figure.update_layout(yaxis_tickformat=',.2%', xaxis_tickformat=',.2%')
figure.update_layout(
    title_text ='Risk vs. Returns',
    yaxis_title ='Returns',
    xaxis_title = 'Standard Deviation')
figure.show()

## Let's say we would like to add manager "x" to our portfolio:

In [10]:
# Let "x" be = 'AMZN':
NM = df['AMZN']

# Let our new allocation to the manager be = $15,000,000. Thus:
wNM = [15000000/(115000000)]
print(wNM)

# Had we had this new manager in our portfolio, how would that have changed our performance?
w1 = [weight * (100000000/(115000000)) for weight in w]
print(w1)

print(np.sum(w1+wNM))

#Thus our portfolio would have been:
Port_Ret1 = Man*w1
Port_Ret1 = Port_Ret1.sum(axis=1)
Port_Ret1 = pd.DataFrame(Port_Ret1)
Port_Ret1.columns = ['Portfolio Return']
Port_Ret1['Portfolio Return'] = Port_Ret1['Portfolio Return'] + (wNM*NM)

Excess2 = ((1+Port_Ret1['Portfolio Return'])/(1+df['IWV']))-1
Tracking_Error_2 = Excess2.std()*np.sqrt(12)
print("Tracking Error:",f"{round(Tracking_Error_2/.0001,2)} bps")

Annualized_Return_2 = ((np.prod(1+Port_Ret1['Portfolio Return']))**(12/((Port_Ret1['Portfolio Return'].count()))))-1
print("Average Return:",f"{round(Annualized_Return_2*100,2)}%")

#The Std Dev is:
Stdev_2 = (Port_Ret1['Portfolio Return'].std()*np.sqrt(12))
print("Stdev:",f"{round(Stdev_2*100,2)}%")

# Benchmark's Annualized Returns:
BMR = ((np.prod(1+df['IWV']))**(12/((df['IWV'].count()))))-1
#Thus:
IR2 = (Annualized_Return_2-BMR)/(Tracking_Error_2)
print("Information Ratio:",f"{round(IR2,2)}")

Risk_Reward2 = Annualized_Return_2/Stdev_2
print("Reward/Risk:",f"{round(Risk_Reward2,2)}")

[0.13043478260869565]
[0.17391304347826086, 0.13043478260869565, 0.43478260869565216, 0.13043478260869565]
1.0
Tracking Error: 1014.17 bps
Average Return: 8.01%
Stdev: 18.63%
Information Ratio: 0.13
Reward/Risk: 0.43


### As a comparison - Adding a New Manager:

In [11]:
Comp2 = ["Annualized Returns", "Standard Deviation", "Information Ratio", "Reward/Risk"]
Comp2 = pd.DataFrame(Comp2)
Comp2['Original Portfolio'] = [Annualized_Return_1, Stdev_1, IR, Risk_Reward]
Comp2['New Portfolio'] = [Annualized_Return_2, Stdev_2, IR2, Risk_Reward2]
Comp2['Changes'] = Comp2['New Portfolio'] - Comp2['Original Portfolio']
Comp2 = Comp2.rename(columns={Comp2.columns[0]: "Metrics"})
Comp2

,Metrics,Original Portfolio,New Portfolio,Changes
0,Annualized Returns,0.072636,0.080135,0.007499
1,Standard Deviation,0.181557,0.186281,0.004724
2,Information Ratio,0.051204,0.131097,0.079893
3,Reward/Risk,0.400072,0.430182,0.030111


### What if rather than adding to the portfolio, we might want to find a replacement for an existing manager?

In [12]:
# In this case we only need to change our "Manager Data":
# Originally, it looked like this: Man = df[['AAPL', 'META', 'JNJ', 'BRK-B']] - Let's say we want to 
# relocate JNJ's weight to AMZN:
Man = df[['AAPL', 'META', 'AMZN', 'BRK-B']]
Port_Ret2 = Man*w
Port_Ret2 = Port_Ret2.sum(axis=1)
Port_Ret2 = pd.DataFrame(Port_Ret2)
Port_Ret2.columns = ['Portfolio Return']
Port_Ret2['Portfolio Return'] = Port_Ret2['Portfolio Return'] + (wNM*NM)
Port_Ret2 = Port_Ret2[1::]
print(w)

Excess3 = ((1+Port_Ret2['Portfolio Return'])/(1+df['IWV']))-1
Tracking_Error_3 = Excess3.std()*np.sqrt(12)
print("Tracking Error:",f"{round(Tracking_Error_3/.0001,2)} bps")

Annualized_Return_3 = ((np.prod(1+Port_Ret2['Portfolio Return']))**(12/((Port_Ret2['Portfolio Return'].count()))))-1
print("Average Return:",f"{round(Annualized_Return_3*100,2)}%")

#The Std Dev is:
Stdev_3 = (Port_Ret2['Portfolio Return'].std()*np.sqrt(12))
print("Stdev:",f"{round(Stdev_3*100,2)}%")

# Benchmark's Annualized Returns:
BMR = ((np.prod(1+df['IWV']))**(12/((df['IWV'].count()))))-1
#Thus:
IR3 = (Annualized_Return_3-BMR)/(Tracking_Error_3)
print("Information Ratio:",f"{round(IR3,2)}")

Risk_Reward3 = Annualized_Return_3/Stdev_3
print("Reward/Risk:",f"{round(Risk_Reward3,2)}")

[0.2, 0.15, 0.5, 0.15]
Tracking Error: 1829.04 bps
Average Return: 10.72%
Stdev: 30.49%
Information Ratio: 0.22
Reward/Risk: 0.35


### Let's Compare again - Replacing a Manager:

In [13]:
Comp3 = ["Annualized Returns", "Standard Deviation", "Information Ratio", "Reward/Risk"]
Comp3 = pd.DataFrame(Comp3)
Comp3['Original Portfolio'] = [Annualized_Return_1, Stdev_1, IR, Risk_Reward]
Comp3['New Portfolio'] = [Annualized_Return_3, Stdev_3, IR3, Risk_Reward3]
Comp3['Changes'] = Comp3['New Portfolio'] - Comp3['Original Portfolio']
Comp3 = Comp3.rename(columns={Comp3.columns[0]: "Metrics"})
Comp3

,Metrics,Original Portfolio,New Portfolio,Changes
0,Annualized Returns,0.072636,0.107214,0.034578
1,Standard Deviation,0.181557,0.304938,0.123381
2,Information Ratio,0.051204,0.220743,0.169539
3,Reward/Risk,0.400072,0.351593,-0.048478


### Portfolio Allocation
###### Long-Only, change bounds to (-1,2) for long-short

In [14]:
# Lets call df the following:
dff = df[['AAPL', 'META', 'JNJ', 'BRK-B', 'AMZN']] # This depends on whether we want to include all of these stocks
dff = dff[1::]
# Let's say our returns are:
df_AvgRet = ((np.prod(1+dff))**(12/((dff.count()))))-1

#The Standard deviation of monthly returns:
df_Stdev = ((dff.std())*(np.sqrt(12)))

# The Variance
df_Var = dff.var()*12

# And the Covariance
df_Cov = dff.cov()*12

### Let's find our Optimal Portfolio

In [15]:
# Let's say that we would like to minimize the standard deviation
def minVar(Weights1):
    W_opt1 = np.array(Weights1)
    # Our returns are the sumproduct of average annualized returns and their respective weight
    Returns_opt1 = np.sum(df_AvgRet * W_opt1)
    # And the standard deviation is the sqrt of the matrix multiplication 
    Stdev_opt1 = np.sqrt(np.dot(W_opt1.T, np.dot(df_Cov, W_opt1)))
    return Stdev_opt1

#Making sure our weights sum to 1
def SumIsOne1(Weights1):
    return np.sum(Weights1)-1

# First let's say we have equal weights
w0 = pd.Series([1/len(dff.columns) for x in range(len(dff.columns))])
# And they can either be 0 or 1
bounds1 = pd.Series([(0,1) for x in range(len(dff.columns))]) #-1,2 for long-short

# Our constraint is that the sum of weights should be equal to 1
constraints1 = ({'type':'eq', 'fun': SumIsOne1})
# Now let's minimize our standard deviation, using our "guess" that the weights are equal, and the bound 0,1
Var1 = minimize(minVar, w0, method='SLSQP', bounds=bounds1,
                       constraints=constraints1)

# Our weights should be:
W = Var1['x']
N = dff.columns
Weights = [W, N]
Weights = pd.DataFrame(Weights).T
Weights.columns = ['Weight', 'Security']
print(round(Weights['Weight'].sum(), 3))
Weights

1.0


,Weight,Security
0,0.0,AAPL
1,0.021686,META
2,0.660341,JNJ
3,0.25549,BRK-B
4,0.062483,AMZN


### Using these weights, we should expect the following:

In [16]:
def opt_sd1(W):
    Stdev_opt = np.sqrt(np.dot(W.T, np.dot(df_Cov, W)))
    return Stdev_opt

def opt_ret1(W):
    Ret_opt = np.sum(df_AvgRet * W)
    return Ret_opt

W = Var1['x']

MV_Sd = opt_sd1(W)
MV_Ret = opt_ret1(W)
MV_Port = [MV_Sd, MV_Ret]
MV_Port = pd.DataFrame(MV_Port).T
MV_Port.columns = ['Std. Dev', 'Returns']

MV_Port

,Std. Dev,Returns
0,0.164911,0.048864


### We could also find our tangent portfolio:
Max Sharpe

In [68]:
# let risk-free (rf) be 
rf = 0.0019
def negShar(Weights1, sign=-1.0):
    W_opt1 = np.array(Weights1)
    Returns_opt1 = np.sum(df_AvgRet * W_opt1)
    Stdev_opt1 = np.sqrt(np.dot(W_opt1.T, np.dot(df_Cov, W_opt1)))
    # Now we should maximize our Sharpe. Thus, minimize -Sharpe
    Sharpe_opt1 = (Returns_opt1)/Stdev_opt1
    return sign*Sharpe_opt1

def SumIsOne1(Weights1):
    return np.sum(Weights1)-1

w0 = pd.Series([1/len(dff.columns) for x in range(len(dff.columns))])
bounds1 = pd.Series([(0,1) for x in range(len(dff.columns))]) ### Change to -1,2 for long-short
constraints1 = ({'type':'eq', 'fun': SumIsOne1})
Sharp_W1 = minimize(negShar, w0, method='SLSQP', bounds=bounds1,
                       constraints=constraints1)


W1 = Sharp_W1['x']
N = dff.columns
Weights1 = [W1, N]
Weights1 = pd.DataFrame(Weights1).T
Weights1.columns = ['Weight', 'Security']
print(round(Weights1['Weight'].sum(), 3))
Weights1

1.0


,Weight,Security
0,1.0,AAPL
1,0.0,META
2,0.0,JNJ
3,0.0,BRK-B
4,0.0,AMZN


### Using these weights, we should expect the following:

In [18]:
def opt_sd(W1):
    Stdev_opt = np.sqrt(np.dot(W1.T, np.dot(df_Cov, W1)))
    return Stdev_opt

def opt_ret(W1):
    Ret_opt = np.sum(df_AvgRet * W1)
    return Ret_opt

W1 = Sharp_W1['x']

S_Sd = opt_sd(W1)
S_Ret = opt_ret(W1)
S_Port = [S_Sd, S_Ret]
S_Port = pd.DataFrame(S_Port).T
S_Port.columns = ['Std. Dev', 'Returns']

S_Port

,Std. Dev,Returns
0,0.312053,0.233977


## Let's now find the Efficient Frontier

In [19]:
# Let the max and min returns equal to:
max_ret1 = df_AvgRet.max()
min_ret1 = df_AvgRet.min()

# Returns num evenly spaced samples, calculated over the interval (3,000)
returns1 = np.linspace(min_ret1, max_ret1,3000)

vol_opt1 = []

def minStdev1(Weights1):
    W_opt1 = np.array(Weights1)
    Stdev_opt1 = np.sqrt(np.dot(W_opt1.T, np.dot(df_Cov, W_opt1)))
    return Stdev_opt1

def Rets1(Weights1):
    W_opt1 = np.array(Weights1)
    Ret = np.sum(df_AvgRet * W_opt1)
    return Ret

for Ret in returns1:
    constraints1 = ({'type':'eq', 'fun':SumIsOne1},
                   {'type':'eq', 'fun': lambda w0: Rets1(w0) - Ret})
    opt2 = minimize(minStdev1, w0, method='SLSQP', bounds=bounds1, constraints=constraints1)
    vol_opt1.append(opt2['fun'])

Vol_Opt1 = np.array(vol_opt1)
Ret1 = np.array(returns1)

Port_Eff = [Vol_Opt1, Ret1]
Port_Eff = pd.DataFrame(Port_Eff).T
Port_Eff.columns = ['Std. Dev', 'Returns']

,Std. Dev,Returns
0,0.352330,-0.109676
1,0.352112,-0.109562
2,0.351894,-0.109447
3,0.351676,-0.109332
4,0.351459,-0.109218
...,...,...
2995,0.311456,0.233518
2996,0.311605,0.233633
2997,0.311754,0.233747
2998,0.311904,0.233862


### And the CAL:

In [20]:
a = Port_Eff
a = pd.DataFrame(a)
rp = S_Port['Returns'].values[0]
sd = S_Port['Std. Dev'].values[0]
a['CAL'] = rf + ((rp-rf)/sd)*a['Std. Dev']

In [21]:
figure = go.Figure()
figure.add_trace(go.Scatter(x=Port_Eff['Std. Dev'], y=Port_Eff['Returns'], name='Efficient Frontier'))
figure.add_trace(go.Scatter(x=MV_Port['Std. Dev'], y=MV_Port['Returns'], name='Minimum Variance'))
figure.add_trace(go.Scatter(x=S_Port['Std. Dev'], y=S_Port['Returns'], name='Tangent Portfolio'))
figure.add_trace(go.Scatter(x=a['Std. Dev'], y=a['CAL'], name='CAL'))
figure.update_layout(yaxis_tickformat=',.2%', xaxis_tickformat=',.2%', title_text ='Portfolio')
figure.update_layout(
    title_text ='Efficient Frontier',
    yaxis_title ='Returns',
    xaxis_title = 'Standard Deviation')
figure.show()

### And the minimum variance weights:

In [22]:
# Since our portfolio could be a long short, a pie chart may not capture negative weights. Use px.bar instead
fig = px.bar(Weights, y='Weight', x='Security',
             title='Min Variance Portfolio Weights')
fig.update_layout(yaxis_tickformat=',.2%')
fig.show()

In [23]:
fig = px.bar(Weights1, y='Weight', x='Security',
             title='Max Sharpe Portfolio Weights')
fig.update_layout(yaxis_tickformat=',.2%')
fig.show()

### Let's see if this portfolio would be better than our current allocation:

In [24]:
# Let's compare the portfolio with AMZN maintaining all other constituents:
# This is:
Port_Ret1

# Now our new optimal portfolio, the Minimum Variance Portfolio:
MV = dff*W
MV = MV.sum(axis=1)
Port_Ret1['Min Var'] = MV

# Thus:
Port_Ret1 = Port_Ret1[1::]

In [25]:
layout = load_figure_template('DARKLY')

figure = go.Figure()
figure.add_trace(go.Scatter(x=Port_Ret1.index, y=Port_Ret1['Portfolio Return'], name='Original Portfolio'))
figure.add_trace(go.Scatter(x=Port_Ret1.index, y=Port_Ret1['Min Var'], name='Minimum Variance'))
figure.update_layout(
    title_text ='Original vs. Minimum Variance Portfolios',
    yaxis_title ='Returns',
    xaxis_title = 'Date')
figure.update_layout(layout)
figure.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=5,
                            label="5Y",
                            step="year",
                            stepmode="backward"),
                        dict(count=3,
                            label="3Y",
                            step="year",
                            stepmode="backward"),
                        dict(count=1,
                            label="YTD",
                            step="year",
                            stepmode="todate"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True,
                ),
                type="date"
            )
        )
figure.update_layout(yaxis_tickformat='.2%')
figure.update_layout(template='plotly_dark',
                  xaxis_rangeselector_font_color='black',
                  xaxis_rangeselector_activecolor='red',
                  xaxis_rangeselector_bgcolor='green',)
figure.update_layout(xaxis=dict(showgrid=False),
                     yaxis=dict(showgrid=False))
figure.show()

In [26]:
# Looking back at our efficient frontier:
std = Port_Ret1['Portfolio Return'].std()*(np.sqrt(12))
ret = ((np.prod(1+Port_Ret1['Portfolio Return']))**(12/((Port_Ret1['Portfolio Return'].count()))))-1
Port_Ret = np.array(ret)
Port_Std = np.array(std)
Port_Metrics = [Port_Ret, Port_Std]
Port_df = pd.DataFrame(Port_Metrics).T
Port_df.columns = ['Returns', 'Std. Dev']

figure = go.Figure()
figure.add_trace(go.Scatter(x=Port_Eff['Std. Dev'], y=Port_Eff['Returns'], name='Efficient Frontier'))
figure.add_trace(go.Scatter(x=MV_Port['Std. Dev'], y=MV_Port['Returns'], name='Minimum Variance'))
figure.add_trace(go.Scatter(x=S_Port['Std. Dev'], y=S_Port['Returns'], name='Tangent Portfolio'))
figure.add_trace(go.Scatter(x=Port_df['Std. Dev'], y=Port_df['Returns'], name='Original Portfolio + AMZN'))
figure.update_layout(yaxis_tickformat=',.2%', xaxis_tickformat=',.2%', title_text ='Portfolio')
figure.update_layout(
    title_text ='Efficient Frontier vs. Current Portfolio',
    yaxis_title ='Returns',
    xaxis_title = 'Standard Deviation')
figure.update_layout(layout)
figure.show()

### And our current allocation - Including new manager:

In [27]:
W = w1+wNM
N = dff.columns
Weights = [W, N]
Weights = pd.DataFrame(Weights).T
Weights.columns = ['Weight', 'Security']
print(round(Weights['Weight'].sum(), 3))
fig = px.pie(Weights, values='Weight', names='Security', labels={'Security':'Security'},
             title='Current Allocation')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(yaxis_tickformat=',.2%')
fig.show()

1.0


_______________________________________________________________________________________________________________________________
## Factor Attribution (Fama-French): Portfolio

<div>
<img src='https://github.com/leofelip/Examples/blob/main/Images/Screenshot%202022-10-08%20111513.png' style="width:80%; padding-right: 20px; padding-left: 20px; text-align:center""/>
    </div>

In [28]:
df_ff3_monthly = gff.carhart4Factor(frequency='m') # You could also use other factors (e.g., 3 Factors, 5 Factors, etc.)
df_ff3_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
df_ff3_monthly.set_index('Date', inplace=True)
FF3 = pd.merge_asof(Port_Ret1['Portfolio Return'], df_ff3_monthly, on='Date')
FF3 = pd.merge_asof(FF3, df['IWV'], on='Date')
FF3 = FF3[1::]
FF3.set_index('Date', inplace=True)
FF3['P-rf'] = ((1+FF3['Portfolio Return'])/(1+ FF3['RF']))-1
FF3['BM-rf'] = ((1+FF3['IWV'])/(1+ FF3['RF']))-1

In [29]:
X = FF3[['BM-rf', 'SMB', 'HML', 'MOM']][3::]
y = FF3['P-rf'][3::]
X = sm.add_constant(X)
ff_model = sm.OLS(y, X).fit()
ff_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   P-rf   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     41.27
Date:                Thu, 06 Oct 2022   Prob (F-statistic):           2.91e-15
Time:                        15:46:45   Log-Likelihood:                 122.08
No. Observations:                  55   AIC:                            -234.2
Df Residuals:                      50   BIC:                            -224.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0022      0.004      0.581      0.564      -0.005       0.010
BM-rf          0.8285      0.067     12.350      0.000       0.694       0.963
SMB           -0.0555      0.142     -0.390      0.699      -0.341       0.230
HML           -0.1910      0.094     -2.038      0.047      -0.379      -0.003
MOM           -0.0601      0.106     -0.569      0.572      -0.272       0.152
==============================================================================
Omnibus:                        1.121   Durbin-Watson:                   2.200
Prob(Omnibus):                  0.571   Jarque-Bera (JB):                1.032
Skew:                           0.151   Prob(JB):                        0.597
Kurtosis:                       2.401   Cond. No.                         41.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Factor Attribution (Fama-French): New Manager

In [30]:
FF3 = pd.merge_asof(FF3, df['AMZN'], on='Date')
FF3['NM-rf'] = ((1+FF3['AMZN'])/(1+FF3['RF']))-1

X = FF3[['BM-rf', 'SMB', 'HML', 'MOM']][3::]
y = FF3['NM-rf'][3::]
X = sm.add_constant(X)
ff_model = sm.OLS(y, X).fit()
ff_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  NM-rf   R-squared:                       0.537
Model:                            OLS   Adj. R-squared:                  0.500
Method:                 Least Squares   F-statistic:                     14.52
Date:                Thu, 06 Oct 2022   Prob (F-statistic):           6.19e-08
Time:                        15:46:45   Log-Likelihood:                 73.515
No. Observations:                  55   AIC:                            -137.0
Df Residuals:                      50   BIC:                            -127.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0003      0.009      0.032      0.975      -0.018       0.018
BM-rf          1.1992      0.162      7.394      0.000       0.873       1.525
SMB           -0.1693      0.344     -0.492      0.625      -0.861       0.522
HML           -0.2330      0.227     -1.028      0.309      -0.688       0.222
MOM           -0.0144      0.255     -0.056      0.955      -0.527       0.498
==============================================================================
Omnibus:                        4.646   Durbin-Watson:                   2.105
Prob(Omnibus):                  0.098   Jarque-Bera (JB):                5.070
Skew:                           0.204   Prob(JB):                       0.0793
Kurtosis:                       4.430   Cond. No.                         41.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

_______________________________________________________________________________________________________________________________
<h1>Changing the Portfolio Beta with Forwards</h1>

<div>
    <h4> From CAPM: </h4>
    </div>
<div>
<img src='https://github.com/leofelip/Examples/blob/main/Images/Screenshot%202022-10-08%20111414.png' style="width:30%; padding-right: 20px; padding-left: 20px; text-align:center"/>
    </div>
<div>
    <h4> Thus: </h4>
    </div>
<div>
<img src='https://github.com/leofelip/Examples/blob/main/Images/Screenshot%202022-10-08%20111433.png' style="width:25%; padding-right: 20px; padding-left: 20px; text-align:center"/>
    </div>
<div>
    <h4> Where </h4>
    </div>
<div>
<img src='https://github.com/leofelip/Examples/blob/main/Images/Screenshot%202022-10-08%20111448.png' style="width:20%; padding-right: 20px; padding-left: 20px; text-align:center"/>
    </div>
    
<div>
    <p> Let's consider using S&P 500 synthetic forwards for this hedge: </p>
    </div>

In [31]:
#Market Yield on U.S. Treasury Securities at 1-Month Constant Maturity, Quoted on an Investment Basis
Rates = 'DGS1MO' #FRED

#S&P 500
SP5 = '^GSPC' #Yahoo! Finance

Rates = web.DataReader(Rates, 'fred', start, end)
Rates.reset_index('DATE', inplace=True, drop=False)
Rates.dropna(inplace=True)
Rates.rename(columns={'DATE':'Date'}, inplace=True)
Rates = Rates.groupby(pd.Grouper(key="Date", freq="MS")).DGS1MO.apply(stats.gmean) # Group by geomean (MS = Month Start)

#S&P 500
SP5 = web.DataReader(SP5, 'yahoo', start, end)
SP5.reset_index('Date', inplace=True, drop=False)
SP5.dropna(inplace=True)
SP5 = SP5[['Date', 'Adj Close']]
SP5.rename(columns={'Adj Close':'Adj_Close_SP5'}, inplace=True)
SP5 = SP5.groupby(pd.Grouper(key="Date", freq="MS")).Adj_Close_SP5.apply(stats.gmean) # Group by geomean (MS = Month Start)
f = pd.merge_asof(Rates, SP5, on='Date')
f['DGS1MO'] = f['DGS1MO']/100

FF3 = pd.merge_asof(FF3, f, on='Date')

# The 1-Month forward price of SPX should be:
# F = S0*e^(rt)
FF3['Fwd'] = FF3['Adj_Close_SP5'] * (e**(FF3['DGS1MO']*(1/12)))

In [32]:
#Our "Beta" (From CAPM, though it could also work for the Fama-French as well):
X = FF3[['BM-rf']][3::]
y = FF3['P-rf'][3::]
X = sm.add_constant(X)
CAPM_model = sm.OLS(y, X).fit()
Beta = CAPM_model.params['BM-rf']
print('Current Portfolio Beta:', round(Beta,2))

Current Portfolio Beta: 0.84


In [33]:
Beta = CAPM_model.params['BM-rf']
PV = 115000000
FF3['P'] = (1+FF3['Portfolio Return'])*PV

# Set new beta to:
Beta_n = 1.20

#Thus:
FF3['H'] = (-P*(Beta-Beta_n))/FF3['Fwd']
# Our "profit" from the hedge would be:
FF3['Hedge']= (FF3['Adj_Close_SP5'] - FF3['Fwd'].shift(1))*FF3['H']
# The hedged portfolio:
FF3['Hedged'] = FF3['Hedge'] + FF3['P']
# Hedged Returns:
FF3['Hedged'] = np.log(FF3['Hedged']) - np.log(FF3['Hedged'].shift(1))
# And our excess return:
FF3['Hedged-rf'] = ((1+FF3['Hedged'])/(1+FF3['RF']))-1

In [34]:
X = FF3[['BM-rf']][3::]
y = FF3['Hedged-rf'][3::]
X = sm.add_constant(X)
ff_model = sm.OLS(y, X).fit()
New_Beta = ff_model.params['BM-rf']
print('New Portfolio Beta:', round(New_Beta,2))

New Portfolio Beta: 1.19


-------------------------------------------------------------------------------------------------------------------------------
###### Leo 